In [1]:
####
import geopandas as gpd
import pandas as pd
import pyarrow.parquet as pq
import os
from shapely.geometry import Point
import subprocess
import multiprocessing
import glob
import sys
from tqdm import tqdm
import shutil
import glob


###data under a folder 
#folder_path = '/gpfs/data1/vclgp/xiongl/IS2global/process/bra_is2_20m/'
folder_path = '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/*/'
### file name is the same under s_000 s_001 s_002 ....
# Get the list of files in the folder
files = glob.glob(os.path.join(folder_path, '*.parquet'))
files[1]

/tmp/ipykernel_71098/941660474.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


'/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b8efffffffff.parquet'

In [2]:
import os

def get_duplicate_basenames(file_paths):
    basename_count = {}
    result_dict = {}

    for file_path in file_paths:
        # Get the basename of the file
        basename = os.path.basename(file_path)

        # If the basename is already in the dictionary, it's a duplicate
        if basename in basename_count:
            # Increment the count for this basename
            basename_count[basename] += 1
            # Add the file path to the dictionary using the basename as the key
            result_dict[basename].append(file_path)
        else:
            # If it's the first occurrence of the basename, initialize the count to 1
            basename_count[basename] = 1
            # Create a list with the file path for this basename
            result_dict[basename] = [file_path]

    # Filter the dictionary to only include basenames that appear more than once
    result_dict = {basename: file_paths for basename, file_paths in result_dict.items() if basename_count[basename] > 1}

    return result_dict



result = get_duplicate_basenames(files)
print(result)


{'8344e0fffffffff.parquet': ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344e0fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/8344e0fffffffff.parquet'], '832773fffffffff.parquet': ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832773fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120W_30N/832773fffffffff.parquet'], '834442fffffffff.parquet': ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/834442fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120W_30N/834442fffffffff.parquet'], '832751fffffffff.parquet': ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832751fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120W_30N/832751fffffffff.parquet'], '8344e6fffffffff.parquet': ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344e6fffffffff.parquet', '/gpfs/

In [3]:
print(len(result))
for key in result:
    value = result[key]
    if (len(value) > 2):
            print(f"The value of '{key}' is {value}.")
            df1 = pd.read_parquet(value[0])
            df2 = pd.read_parquet(value[1])
            df3 = pd.read_parquet(value[2])
            #df4 = pd.read_parquet(value[3])
            break

805
The value of '834455fffffffff.parquet' is ['/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/834455fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120W_30N/834455fffffffff.parquet', '/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/834455fffffffff.parquet'].


In [4]:
print(len(df1),len(df2),len(df3) )

158575 1160 7874


In [5]:
# data file size
#df = pd.read_parquet('/gpfs/data1/vclgp/xiongl/IS2global//data/h3/atl08/shots_hex/834455fffffffff.parquet')

In [6]:
#print(len(df))

In [9]:
# merge not same 
import pyarrow as pa
import pyarrow.parquet as pq
def stream_write_parquet(ofile, flist, hres=None, check_shots=False): 
    shots = []
    pqwriter = None    
    for i,f in enumerate(flist): ## index i, file f  ## (ftmp, [upath, new_file], check_shots=True)

        df = pd.read_parquet(f)  ## read all files in old path, new path, but same h3_03 index name by file,
        if hres is not None:
            df = df.set_index(hres)
        
        if check_shots: ### merge data if files exist. 
            # df = df.drop_duplicates('shot_number')
            df = df[~df.shot_number.isin(shots)]
            shots += df.shot_number.to_list()
            #print('df before')
            #print(df)
            #df = df[~df['land_segments/delta_time'].isin(shots)] ## shots means  delta_time ??? ### new df exclude in shots list.
            #shots += df['land_segments/delta_time'].to_list() ## save shots to a list. 
            #print('df after')
            #print(df)
        if len(df) == 0:
            continue

        table = pa.Table.from_pandas(df)
        if i == 0:
            pqwriter = pq.ParquetWriter(ofile, table.schema) ### pyarrow parquet 
        pqwriter.write_table(table) ##### write_table;  seems this step will merge 
    if bool(pqwriter):
        pqwriter.close()




['/gpfs/data1/vclgp/xiongl/IS2global/process/global_bash/sub_90W_30N/834455fffffffff.parquet',
 '/gpfs/data1/vclgp/xiongl/IS2global/process/global_bash/sub_120W_30N/834455fffffffff.parquet',
 '/gpfs/data1/vclgp/xiongl/IS2global/process/global_bash/sub_90W_0N/834455fffffffff.parquet']

In [10]:
for key in result:
    print('merging file: ', key)
    value = result[key]
    ofile='/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/' + key
    flist = value
    stream_write_parquet(ofile, flist, hres=None, check_shots=True)

merging file:  8344e0fffffffff.parquet
merging file:  832773fffffffff.parquet
merging file:  834442fffffffff.parquet
merging file:  832751fffffffff.parquet
merging file:  8344e6fffffffff.parquet
merging file:  83275bfffffffff.parquet
merging file:  832776fffffffff.parquet
merging file:  8344e4fffffffff.parquet
merging file:  831b93fffffffff.parquet
merging file:  83272dfffffffff.parquet
merging file:  83272cfffffffff.parquet
merging file:  832b24fffffffff.parquet
merging file:  83445bfffffffff.parquet
merging file:  832671fffffffff.parquet
merging file:  834458fffffffff.parquet
merging file:  832b2cfffffffff.parquet
merging file:  8344f6fffffffff.parquet
merging file:  832675fffffffff.parquet
merging file:  8344f0fffffffff.parquet
merging file:  832b29fffffffff.parquet
merging file:  832643fffffffff.parquet
merging file:  8344e5fffffffff.parquet
merging file:  832754fffffffff.parquet
merging file:  83445cfffffffff.parquet
merging file:  832644fffffffff.parquet
merging file:  8344f5ffff

merging file:  833c4bfffffffff.parquet
merging file:  83309cfffffffff.parquet
merging file:  833c72fffffffff.parquet
merging file:  8340eefffffffff.parquet
merging file:  833d4efffffffff.parquet
merging file:  8340e6fffffffff.parquet
merging file:  833d6dfffffffff.parquet
merging file:  833d5afffffffff.parquet
merging file:  833d5bfffffffff.parquet
merging file:  83401efffffffff.parquet
merging file:  832530fffffffff.parquet
merging file:  833012fffffffff.parquet
merging file:  83311bfffffffff.parquet
merging file:  833083fffffffff.parquet
merging file:  833080fffffffff.parquet
merging file:  83311cfffffffff.parquet
merging file:  8340adfffffffff.parquet
merging file:  834196fffffffff.parquet
merging file:  83158efffffffff.parquet
merging file:  83309efffffffff.parquet
merging file:  8325a5fffffffff.parquet
merging file:  832584fffffffff.parquet
merging file:  8340e1fffffffff.parquet
merging file:  833d6cfffffffff.parquet
merging file:  832591fffffffff.parquet
merging file:  833120ffff

merging file:  833975fffffffff.parquet
merging file:  833971fffffffff.parquet
merging file:  83bc0afffffffff.parquet
merging file:  83bc19fffffffff.parquet
merging file:  834240fffffffff.parquet
merging file:  833c23fffffffff.parquet
merging file:  834399fffffffff.parquet
merging file:  834271fffffffff.parquet
merging file:  83438bfffffffff.parquet
merging file:  834352fffffffff.parquet
merging file:  834225fffffffff.parquet
merging file:  833cf3fffffffff.parquet
merging file:  834241fffffffff.parquet
merging file:  83616cfffffffff.parquet
merging file:  834274fffffffff.parquet
merging file:  833c26fffffffff.parquet
merging file:  833c05fffffffff.parquet
merging file:  83424bfffffffff.parquet
merging file:  833c2afffffffff.parquet
merging file:  83846dfffffffff.parquet
merging file:  833c2bfffffffff.parquet
merging file:  834244fffffffff.parquet
merging file:  834230fffffffff.parquet
merging file:  834220fffffffff.parquet
merging file:  833d13fffffffff.parquet
merging file:  83424effff

merging file:  8366dcfffffffff.parquet
merging file:  8366c3fffffffff.parquet
merging file:  8366cdfffffffff.parquet
merging file:  838ac2fffffffff.parquet
merging file:  838fa6fffffffff.parquet
merging file:  838a5afffffffff.parquet
merging file:  838f2cfffffffff.parquet
merging file:  8366d1fffffffff.parquet
merging file:  8366defffffffff.parquet
merging file:  8366ebfffffffff.parquet
merging file:  838aeafffffffff.parquet
merging file:  838a49fffffffff.parquet
merging file:  838ac5fffffffff.parquet
merging file:  8366d3fffffffff.parquet
merging file:  838a59fffffffff.parquet
merging file:  838f21fffffffff.parquet
merging file:  8366cefffffffff.parquet
merging file:  838a4bfffffffff.parquet
merging file:  838ae9fffffffff.parquet
merging file:  838ac0fffffffff.parquet
merging file:  838ae8fffffffff.parquet
merging file:  838a5bfffffffff.parquet
merging file:  8366ccfffffffff.parquet
merging file:  838a4afffffffff.parquet
merging file:  835f29fffffffff.parquet
merging file:  835f55ffff

In [11]:
len(pd.read_parquet(ofile))

308539

In [12]:
import os
import shutil

def move_files(files, destination_folder):
    # Get a list of files in the source folder
    

    for file_path in files:
        base = os.path.basename(file_path)
        destination_path = os.path.join(destination_folder, base)

        if not os.path.exists(destination_path):
            shutil.move(file_path, destination_folder)
            print(f"Moved {file_path} to {destination_folder}.")
        else:
            print(f"{file_path} already exists in {destination_folder}.")

# Example usage:
destination_folder = "/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/"

move_files(files, destination_folder)


Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/834450fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b8efffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832759fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832771fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344c6fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832af2fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_ba

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/830eedfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/830e4efffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b35fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/83445bfffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344cbfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/830ecefffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/83266dfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344f1fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344d5fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832aa8fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832770fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344cdfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_ba

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/830ecdfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344cefffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832a16fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b85fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b99fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/83265dfffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832664fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832a86fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832758fffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832b18fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/8344d0fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_30N/832a14fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/

...

/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/8344f5fffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/835f54fffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/834454fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/836676fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/8366eafffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/836649fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/glob

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/83675efffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/8366c0fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/835f46fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/83451bfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/83661afffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_90W_0N/836643fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub

/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/8312d1fffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/832815fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/8328f5fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/8312d2fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/83129cfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_150W_30N/832833fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/g

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/837356fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/83686efffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/836851fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/83696cfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/836856fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/836844fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_ba

Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/834b09fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
/gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/83724dfffffffff.parquet already exists in /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/8368e5fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/836854fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/836941fffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/sub_120E_0N/834b9cfffffffff.parquet to /gpfs/data1/vclgp/xiongl/GEDIglobal/global/global_bash/.
Moved /gpfs/data1/vclgp/xiongl/GEDIglobal/global/

In [13]:
import os

def remove_subfolders():
    current_directory = destination_folder
    subfolders = [folder for folder in os.listdir(current_directory) if 'sub' in folder]
    #print(subfolders )
    for folder in subfolders:
        folder_path = os.path.join(current_directory, folder)
        try:
            shutil.rmtree(folder_path)
            print(f"Removed subfolder: {folder}")
        except OSError as e:
            print(f"Failed to remove subfolder '{folder}': {e}")


remove_subfolders()


Removed subfolder: sub_90W_30N
Removed subfolder: sub_30E_30N
Removed subfolder: sub_120W_60S
Removed subfolder: sub_30E_30S
Removed subfolder: sub_180W_60S
Removed subfolder: sub_30W_30S
Removed subfolder: sub_150W_30S
Removed subfolder: sub_90E_30N
Removed subfolder: sub_0E_30S
Removed subfolder: sub_0E_0N
Removed subfolder: sub_180W_30S
Removed subfolder: sub_30W_60S
Removed subfolder: sub_30W_30N
Removed subfolder: sub_30E_60S
Removed subfolder: sub_60E_0N
Removed subfolder: sub_90W_60S
Removed subfolder: sub_60E_60S
Removed subfolder: sub_150E_0N
Removed subfolder: sub_120W_0N
Removed subfolder: sub_120W_30N
Removed subfolder: sub_180W_30N
Removed subfolder: sub_120E_30N
Removed subfolder: sub_60W_30S
Removed subfolder: sub_0E_60S
Removed subfolder: sub_30W_0N
Removed subfolder: sub_60W_60S
Removed subfolder: sub_30E_0N
Removed subfolder: sub_90W_30S
Removed subfolder: sub_60W_0N
Removed subfolder: sub_90E_60S
Removed subfolder: sub_90E_30S
Removed subfolder: sub_90E_0N
Removed su

In [15]:
os.listdir(destination_folder)[-20:-1]

['833e69fffffffff.parquet',
 '836081fffffffff.parquet',
 '832602fffffffff.parquet',
 '836093fffffffff.parquet',
 '838e25fffffffff.parquet',
 '834095fffffffff.parquet',
 '83598efffffffff.parquet',
 '83b9b4fffffffff.parquet',
 '8320a5fffffffff.parquet',
 '83396dfffffffff.parquet',
 '83ad12fffffffff.parquet',
 '8376a4fffffffff.parquet',
 '837518fffffffff.parquet',
 '83544efffffffff.parquet',
 '83405bfffffffff.parquet',
 '833e53fffffffff.parquet',
 '833d1bfffffffff.parquet',
 '83c2ddfffffffff.parquet',
 '834b35fffffffff.parquet']